In [35]:
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from time import sleep
import json


In [36]:
username = 'sheryl.ter34@gmail.com'
password = 'Blueocean-299'

In [37]:
options = uc.ChromeOptions()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
options.add_experimental_option('prefs', {'intl.accept_languages': 'es'})
driver = uc.Chrome(headless=True, options=options)#, version_main=120)
driver.get('https://www.facebook.com/')

# Se logea
WebDriverWait(
    driver, 60).until(expected_conditions.presence_of_element_located(
        (By.ID, 'email'))).send_keys(username)
driver.find_element(By.ID, 'pass').send_keys(password)
driver.get_screenshot_as_file('screenshot1.png')
WebDriverWait(
    driver, 60).until(expected_conditions.element_to_be_clickable(
        (By.XPATH, '//button[@type="submit"]'))).click()
driver.get_screenshot_as_file('screenshot2.png')
with open('texto.html', 'w') as html_file:
    html_file.write(driver.page_source)
# Espera a que abra la página logueada
WebDriverWait(
    driver, 60).until(expected_conditions.presence_of_element_located(
        (By.XPATH, '//span[text()="Grupos"]')))


# Abre el grupo
driver.set_window_size(1600, 8000)
#driver.get('https://www.facebook.com/groups/157087343117679/?ref=share&mibextid=FvvcmW')
driver.get('https://www.facebook.com/profile.php?id=100067031089582')
sleep(10)
ActionChains(driver).send_keys(Keys.ESCAPE).perform()
# images = WebDriverWait(driver, 180).until(expected_conditions.presence_of_all_elements_located(
#         (By.XPATH, '//img[@height="18"]')))
sleep(60)
images = driver.find_elements(By.XPATH, '//img[@height="18"]')
driver.get_screenshot_as_file('screenshot3.png')
print(len(images))
for image in images:
    ActionChains(driver).move_to_element(image).perform()
    sleep(5)
comments_to_hover = driver.find_elements(By.XPATH, '//span[contains(text(), "comentarios")]')
print(len(comments_to_hover))

for comment in comments_to_hover:
    ActionChains(driver).move_to_element(comment).perform()
    sleep(5)



logs = driver.get_log("performance")
events = [json.loads(entry['message'])['message'] for entry in logs]
events = [event for event in events if 'method' in event and 'Network.response' in event['method']]
bodies = []

# data = [driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': event["params"]["requestId"]}) for event in events]
for event in events:
    # print(event["params"]["requestId"])
    # print(driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': event["params"]["requestId"]}))
    try:
        body = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': event["params"]["requestId"]})
        body["body"] = json.loads(body["body"])
        if 'feedback' in body["body"]["data"] and ('reactors' in body["body"]["data"]["feedback"] or 'commenters' in body["body"]["data"]["feedback"]):
            bodies.append(body)
    except:
        pass

driver.quit()

4
2


In [38]:
bodies[8]

{'base64Encoded': False,
 'body': {'data': {'feedback': {'total_comment_count': 3,
    'commenters': {'count': 3,
     'edges': [{'node': {'__typename': 'User',
        'name': 'Giorgina González Aguilar',
        'id': '815737235'}},
      {'node': {'__typename': 'User',
        'name': 'Reiny Rodriguez',
        'id': '100006656836743'}},
      {'node': {'__typename': 'User',
        'name': 'Jorge Luis González Aguilar',
        'id': '624217742'}}]},
    'id': 'ZmVlZGJhY2s6NjgxNTQ4ODY3NDIyODEw'}},
  'extensions': {'is_final': True}}}

In [39]:
reactions = set()
for body in bodies:
    if 'reactors' in body["body"]["data"]["feedback"]:
        for node in body["body"]["data"]["feedback"]["reactors"]["nodes"]:
            reactions.add((node["id"], node["name"], body['body']['data']['feedback']['id']))
    if 'commenters' in body["body"]["data"]["feedback"]:
        for edge in body["body"]["data"]["feedback"]["commenters"]["edges"]:
            reactions.add((edge["node"]["id"], edge["node"]["name"],
                          body['body']['data']['feedback']['id']))

reactions

{('100000883198689',
  'Carmen Aguilar de González',
  'ZmVlZGJhY2s6NjU3NTExNDYzMTU5ODg0'),
 ('100004933322691',
  'Carlos Julio Chirinos Delgado',
  'ZmVlZGJhY2s6NjgxNTQ4ODY3NDIyODEw'),
 ('100006656836743', 'Reiny Rodriguez', 'ZmVlZGJhY2s6NjgxNTQ4ODY3NDIyODEw'),
 ('100067031089582', 'GioTest', 'ZmVlZGJhY2s6NjU3NTExNDYzMTU5ODg0'),
 ('1021676186', 'Juan Vasquez', 'ZmVlZGJhY2s6NjgxNTQ4ODY3NDIyODEw'),
 ('624217742',
  'Jorge Luis González Aguilar',
  'ZmVlZGJhY2s6NjgxNTQ4ODY3NDIyODEw'),
 ('815737235',
  'Giorgina González Aguilar',
  'ZmVlZGJhY2s6NjgxNTQ4ODY3NDIyODEw')}